# User Ingestor

Upload users to DHIS2 via CSV. The app validates your file first, then uploads if everything looks good. If there are errors, you'll get a report to fix and retry.

---

### How it works
1. Pick a CSV file
2. App checks every row against the rules below
3. All rows valid? --> uploads to API automatically
4. Any row invalid? --> gives you an error report (no upload happens)

---

### CSV format

Your CSV needs **exactly these 16 columns** (no extra, no missing):

`username`, `password`, `name`, `mobile_number`, `gender`, `date_of_birth`, `email`, `correspondence_address`, `roles`, `employment_type`, `date_of_joining`, `department`, `designation`, `campaign_name`, `administrative_area`, `boundary_code`

---

### What gets validated

| Column | Required? | Must be unique? | Rules |
|--------|-----------|-----------------|-------|
| `username` | Yes | Yes | Letters, numbers, hyphens, underscores. No `/` or `\`. Max 100 chars. |
| `password` | Yes | No | Needs at least one uppercase letter. Max 100 chars. |
| `name` | Yes | No | Only letters and spaces. Max 200 chars. |
| `mobile_number` | No | Yes | 8-15 digits only. |
| `gender` | No | No | `MALE`, `FEMALE`, or `OTHER` |
| `date_of_birth` | No | No | `DD/MM/YYYY` or `DD-MM-YYYY` |
| `email` | No | No | Standard email format. Max 200 chars. |
| `correspondence_address` | No | No | Any text. Max 500 chars. |
| `roles` | Yes | No | Comma-separated. Must match roles in `rolesmapping.json`. |
| `employment_type` | Yes | No | `TEMPORARY` or `PERMANENT` |
| `date_of_joining` | Yes | No | `DD/MM/YYYY` or `DD-MM-YYYY`. Must be a real date. |
| `department` | Yes | No | Must be `OTHER` |
| `designation` | No | No | Any text. Max 100 chars. |
| `campaign_name` | Yes | No | Must be `LLIN-bi` |
| `administrative_area` | Yes | No | Must match a name in `boundary_template.csv` |
| `boundary_code` | Yes | No | Must match an ID in `boundary_template.csv` |

### Other checks
- **boundary_code + administrative_area** must match each other in `boundary_template.csv`
- **username** must be unique within the CSV
- **mobile_number** must be unique within the CSV (blanks are fine)

---

### To get started: Run all cells below (Cell > Run All)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd
import os
from datetime import datetime
import sys
import glob
import time

# Reload modules so code changes take effect without restarting the kernel
import importlib
if 'utils' in sys.modules:
    importlib.reload(sys.modules['utils.api_client'])
    importlib.reload(sys.modules['utils.validators'])
    importlib.reload(sys.modules['utils.user_custom_validators'])
    importlib.reload(sys.modules['utils'])

from utils import create_user_validator
from utils.api_client import APIClient

# Start fresh
for file in glob.glob("uploads/*"):
    if os.path.isfile(file):
        os.remove(file)
print(" All libraries loaded successfully!")
print(" Uploads folder cleared!")


In [ ]:
# --- Config inputs ---
api_url_input = widgets.Text(
    value='http://hcm-moz-impl.abuja:8080/hcm-moz-impl/v1/dhis2/users/ingest?source=EXCEL',
    placeholder='Enter API URL',
    description='API URL:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='800px')
)
tenant_id_input = widgets.Text(
    value='bi',
    placeholder='Enter Tenant ID',
    description='Tenant ID:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)
auth_token_input = widgets.Text(
    value='ee36fdd7-64e7-4583-9c16-998479ff53c0',
    placeholder='Enter Auth Token',
    description='Auth Token:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px')
)

# --- File browser ---
def scan_csv_files():
    csv_files = []
    for pattern in ['uploads/*.csv']:
        csv_files.extend(glob.glob(pattern))
    return sorted(set(csv_files))

def refresh_file_list(b=None):
    csv_files = scan_csv_files()
    if csv_files:
        file_dropdown.options = [('-- Select a CSV file --', '')] + [(f, f) for f in csv_files]
    else:
        file_dropdown.options = [('No CSV files found', '')]
    file_dropdown.value = ''

file_dropdown = widgets.Dropdown(
    options=[('-- Click Refresh to scan --', '')],
    value='',
    description='Select File:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='450px')
)

refresh_button = widgets.Button(
    description='Refresh',
    button_style='info',
    icon='refresh',
    layout=widgets.Layout(width='100px')
)
refresh_button.on_click(refresh_file_list)

file_path_input = widgets.Text(
    value='',
    placeholder='Or type path manually: templates/users.csv',
    description='Manual Path:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='450px')
)

process_button = widgets.Button(
    description=' Process & Upload',
    button_style='primary',
    icon='upload',
    layout=widgets.Layout(width='250px', height='50px')
)

process_output = widgets.Output()
status_label = widgets.HTML(value="<h3 style='color: #3498DB;'> Ready to process CSV</h3>")
summary_data = None

# --- Helpers ---
def clear_uploads_folder(exclude_file=None):
    """Remove old files from uploads/, keep the one we're about to process."""
    import glob
    for file in glob.glob("uploads/*"):
        if os.path.isfile(file):
            if exclude_file and os.path.samefile(file, exclude_file):
                continue
            os.remove(file)

def resolve_file_path(path):
    path = path.strip()
    if os.path.isabs(path):
        return path
    return os.path.abspath(path)

def normalize_date(date_str):
    """Convert DD-MM-YYYY to DD/MM/YYYY so the API accepts it."""
    if pd.isna(date_str) or str(date_str).strip() == '' or str(date_str).lower() == 'nan':
        return date_str
    return str(date_str).strip().replace('-', '/')

def normalize_row_dates(row):
    for col in ['date_of_joining', 'date_of_birth']:
        if col in row.index:
            row[col] = normalize_date(row[col])
    return row

# --- Main handler ---
def on_process_click(b):
    global summary_data
    
    with process_output:
        clear_output()
    
    # Pick file from dropdown or manual input
    input_path = file_dropdown.value if file_dropdown.value else file_path_input.value.strip()
    
    if not input_path:
        status_label.value = "<h3 style='color: red;'> Please select a CSV file or enter a path!</h3>"
        return
    
    csv_path = resolve_file_path(input_path)
    
    if not os.path.exists(csv_path):
        status_label.value = f"<h3 style='color: red;'> File not found: {input_path}</h3>"
        with process_output:
            print(f"[ERROR] File not found")
            print(f"   Input path: {input_path}")
            print(f"   Resolved to: {csv_path}")
            print(f"\n[TIP] Click 'Refresh' to scan for available CSV files")
        return
    
    process_button.disabled = True
    status_label.value = "<h3 style='color: orange;'> Processing...</h3>"
    
    try:
        uploaded_filename = os.path.basename(csv_path)
        upload_path = f"uploads/{uploaded_filename}"
        
        # If file is already in uploads/, use it directly; otherwise copy it there
        source_in_uploads = csv_path.replace('\\', '/').find('/uploads/') != -1 or csv_path.startswith('uploads/')
        
        if source_in_uploads:
            clear_uploads_folder(exclude_file=csv_path)
            upload_path = csv_path
        else:
            clear_uploads_folder()
            import shutil
            shutil.copy(csv_path, upload_path)
        
        # --- Phase 1: Validation ---
        with process_output:
            print("=" * 70)
            print("[PHASE 1] CSV VALIDATION")
            print("=" * 70)
            print(f"\n[INFO] File: {uploaded_filename}")
            print(f"[INFO] Source path: {csv_path}")
            print(f"[INFO] Working copy: {upload_path}")
        
        validator = create_user_validator()
        
        with process_output:
            print(f"\n[VALIDATING] Running validation checks...")
        
        validated_df, summary = validator.validate_csv(upload_path)
        
        with process_output:
            print(f"\n{'=' * 70}")
            print("[VALIDATION SUMMARY]")
            print(f"{'=' * 70}")
            print(f"[HEADER STATUS] {summary['header_status']}")
            if summary['header_status'] == 'ERROR':
                print(f"[HEADER ERROR] {summary['header_message']}")
            print(f"[TOTAL USERS] {summary['total_users']}")
            print(f"[ VALID USERS] {summary['correct_users']}")
            print(f"[ INVALID USERS] {summary['error_users']}")
            print(f"{'=' * 70}\n")
        
        if summary['error_users'] > 0 or summary['header_status'] == 'ERROR':
            # Validation failed - save error report
            base_name = uploaded_filename.rsplit('.', 1)[0]
            error_report_path = f"uploads/{base_name}_errors.csv"
            
            error_df = validated_df[validated_df['validation_status'] == 'ERROR']
            error_df.to_csv(error_report_path, index=False)
            
            with process_output:
                print(f"[FAILED]  Validation failed with {summary['error_users']} errors\n")
                print(f" Updating error report: {error_report_path}")
                print(f"    Error report saved successfully!")
            
            status_label.value = f"<h3 style='color: red;'> Validation Failed: {summary['error_users']} errors. Run the Summary cell below!</h3>"
            
            summary_data = {
                'status': 'FAILED',
                'summary': summary,
                'error_report': error_report_path
            }
            
        else:
            # All good - upload to API
            with process_output:
                print(f"[SUCCESS]  All validations passed!\n")
                print(f"{'=' * 70}")
                print(f"[PHASE 2] API UPLOAD ")
                print(f"{'=' * 70}\n")
            
            validated_df.to_csv(upload_path, index=False)
            
            api_url = api_url_input.value
            tenant_id = tenant_id_input.value
            auth_token = auth_token_input.value
            
            with process_output:
                print(f"[UPLOADING] User Data")
                print(f"   Total Users: {summary['total_users']}")
                print(f"   API URL: {api_url}")
                print(f"   Tenant ID: {tenant_id}")
                print(f"{'=' * 70}\n")
            
            client = APIClient(api_url, tenant_id, auth_token)
            
            base_name = uploaded_filename.rsplit('.', 1)[0]
            final_report_path = f"uploads/{base_name}_result.csv"
            
            df = pd.read_csv(upload_path)
            df['api_status'] = ''
            df['api_status_code'] = ''
            df['api_message'] = ''
            
            success_count = 0
            failed_count = 0
            skipped_count = 0
            
            for idx, row in df.iterrows():
                if row.get('validation_status') == 'CORRECT':
                    with process_output:
                        print(f"[PROCESSING] Row {idx + 1}/{len(df)}: {row.get('username', 'N/A')}", end='')
                    
                    row_normalized = normalize_row_dates(row.copy())
                    
                    # Upload each row as a single-row CSV
                    temp_file = f"temp_upload_{idx}.csv"
                    single_row_df = pd.DataFrame([row_normalized])
                    single_row_df.to_csv(temp_file, index=False)
                    
                    result = client.upload_file(temp_file)
                    
                    df.at[idx, 'api_status'] = result['status']
                    df.at[idx, 'api_status_code'] = str(result['status_code'])
                    df.at[idx, 'api_message'] = result['message']
                    
                    if result['status'] == 'SUCCESS':
                        success_count += 1
                        with process_output:
                            print(f" -> SUCCESS (Status: {result['status_code']})")
                    else:
                        failed_count += 1
                        with process_output:
                            print(f" -> FAILED (Status: {result['status_code']})")
                    
                    if os.path.exists(temp_file):
                        os.remove(temp_file)
                    
                    time.sleep(5)
                else:
                    df.at[idx, 'api_status'] = 'SKIPPED'
                    df.at[idx, 'api_status_code'] = 'N/A'
                    df.at[idx, 'api_message'] = 'Validation failed'
                    skipped_count += 1
            
            df.to_csv(final_report_path, index=False)
            
            with process_output:
                print(f"\n{'=' * 70}")
                print("[API UPLOAD SUMMARY]")
                print(f"{'=' * 70}")
                print(f"[TOTAL UPLOADED] {len(df)}")
                print(f"[SUCCESS] {success_count}")
                print(f"[FAILED] {failed_count}")
                print(f"[SKIPPED] {skipped_count}")
                print(f"{'=' * 70}\n")
                print(f" Updating result file: {final_report_path}")
                print(f"    Result file updated successfully!")
                print(f"    Updated {len(df)} rows\n")
                print(" DATA UPLOAD COMPLETED!")
            
            status_label.value = f"<h3 style='color: green;'> Complete! {success_count} success, {failed_count} failed. Run Summary cell below!</h3>"
            
            summary_data = {
                'status': 'SUCCESS',
                'summary': summary,
                'success_count': success_count,
                'failed_count': failed_count,
                'final_report': final_report_path
            }
        
    except Exception as e:
        with process_output:
            print(f"\n Error: {str(e)}")
            import traceback
            traceback.print_exc()
        status_label.value = f"<h3 style='color: red;'> Process failed: {str(e)}</h3>"
    
    finally:
        process_button.disabled = False

process_button.on_click(on_process_click)
refresh_file_list()

# --- Layout ---
display(HTML("<hr><h2 style='color: #2E86C1;'> Configuration</h2>"))
display(widgets.VBox([
    api_url_input,
    widgets.HBox([tenant_id_input, auth_token_input]),
]))
display(HTML("<hr><h2 style='color: #2E86C1;'> CSV File</h2>"))
display(HTML("<p style='color: #666;'>Select from dropdown or type path manually:</p>"))
display(widgets.HBox([file_dropdown, refresh_button]))
display(file_path_input)
display(process_button)
display(status_label)
display(HTML("<hr><h2 style='color: #2E86C1;'> Processing Logs</h2>"))
display(process_output)
print("\n Application ready! Select a CSV file and click 'Process & Upload'")


---
##  Summary Report & Downloads

**Run the cell below after processing to see results:**

In [ ]:
if 'summary_data' in globals() and summary_data:
    timestamp_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    if summary_data['status'] == 'FAILED':
        summary = summary_data['summary']
        error_report = summary_data['error_report']
        
        display(HTML(f"""
        <div style='background-color: #FADBD8; padding: 20px; border-radius: 10px; border-left: 5px solid #E74C3C; margin: 20px 0;'>
            <h2 style='color: #E74C3C; margin-top: 0;'> VALIDATION FAILED</h2>
            <table style='width: 100%; border-collapse: collapse; background-color: white; margin-top: 15px;'>
                <tr style='background-color: #E74C3C; color: white;'>
                    <th style='padding: 10px; text-align: left;'>Metric</th>
                    <th style='padding: 10px; text-align: center;'>Count</th>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Total Users</td>
                    <td style='padding: 8px; border: 1px solid #ddd; text-align: center; font-weight: bold;'>{summary['total_users']}</td>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'> Valid Users</td>
                    <td style='padding: 8px; border: 1px solid #ddd; text-align: center; color: green; font-weight: bold;'>{summary['correct_users']}</td>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'> Invalid Users</td>
                    <td style='padding: 8px; border: 1px solid #ddd; text-align: center; color: red; font-weight: bold;'>{summary['error_users']}</td>
                </tr>
            </table>
            <p style='margin-top: 15px; font-weight: bold;'> Fix errors and re-upload</p>
        </div>
        <h3 style='color: #E74C3C;'> Download Error Report:</h3>
        <div style="padding: 10px; background-color: white; border-radius: 5px; border: 1px solid #E74C3C; display: inline-block; margin-top: 10px;">
            <a href="{error_report}" download="{os.path.basename(error_report)}"
               style="display: inline-block; padding: 10px 20px; background-color: #E74C3C; color: white;
                      text-decoration: none; border-radius: 5px; font-weight: bold; font-size: 14px;">
                 Download Error Report ({os.path.basename(error_report)})
            </a>
        </div>
        """))
        
    elif summary_data['status'] == 'SUCCESS':
        summary = summary_data['summary']
        success_count = summary_data['success_count']
        failed_count = summary_data['failed_count']
        final_report = summary_data['final_report']
        
        display(HTML(f"""
        <div style="font-family: Arial, sans-serif; padding: 20px; border: 2px solid #007bff; border-radius: 10px; background-color: #f8f9fa; margin: 20px 0;">
            <h2 style="color: #007bff; margin-top: 0;"> Data Upload Summary Report</h2>
            <p style="color: #666; margin-bottom: 10px;">Generated: {timestamp_str}</p>
            
        
            
            <div style="display: flex; justify-content: space-around; margin: 20px 0; flex-wrap: wrap;">
                <div style="text-align: center; padding: 15px; background-color: #d4edda; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
                    <div style="font-size: 32px; font-weight: bold; color: #155724;">{success_count}</div>
                    <div style="color: #155724;">Success</div>
                </div>
                <div style="text-align: center; padding: 15px; background-color: #f8d7da; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
                    <div style="font-size: 32px; font-weight: bold; color: #721c24;">{failed_count}</div>
                    <div style="color: #721c24;">Failed</div>
                </div>
            </div>
        </div>
        """))
        
        status_badge = ' ALL SUCCESS' if failed_count == 0 else ' HAS ERRORS'
        badge_color = '#28a745' if failed_count == 0 else '#dc3545'
        
        display(HTML(f"""
        <div style="margin: 20px 0; padding: 15px; background-color: #e7f3ff; border-left: 4px solid #007bff; border-radius: 5px;">
            <h3 style="margin-top: 0; color: #004085;"> Result CSV</h3>
            <p style="color: #004085; margin-bottom: 15px;">
                The CSV now has <b>api_status</b>, <b>api_status_code</b>, and <b>api_message</b> columns:
            </p>
            <div style="padding: 10px; background-color: white; border-radius: 5px; border: 1px solid #ddd;">
                <div style="display: flex; justify-content: space-between; align-items: center; flex-wrap: wrap;">
                    <div style="flex: 1; min-width: 300px;">
                        <strong style="color: #007bff;"> {os.path.basename(final_report)}</strong>
                        <span style="background-color: {badge_color}; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px; margin-left: 10px;">{status_badge}</span>
                        <br>
                    </div>
                    <a href="{final_report}" download="{os.path.basename(final_report)}" 
                       style="display: inline-block; padding: 8px 16px; background-color: #007bff; color: white; 
                              text-decoration: none; border-radius: 5px; font-weight: bold; font-size: 14px; margin-top: 5px;">
                         Download Result File
                    </a>
                </div>
            </div>
            <p style="color: #004085; font-size: 12px; margin-top: 15px; border-top: 1px solid #bee5eb; padding-top: 10px;">
                <b> Status Values in CSV:</b>
            </p>
            <ul style="font-size: 12px; color: #004085; margin: 5px 0;">
                <li><span style="color: green; font-weight: bold;">SUCCESS:</span> User created successfully (or updated if already exists)</li>
                <li><span style="color: red; font-weight: bold;">FAILED:</span> API request failed - check api_message column for details</li>
                <li><span style="color: gray; font-weight: bold;">SKIPPED:</span> Validation failed (not uploaded)</li>
            </ul>
        </div>
        """))

else:
    print(" No results yet. Run the cell above first to process a CSV.")
